# Player's target share in a given year

### Libraries

In [2]:
import nfl_data_py as nfl
from datetime import datetime
import pandas as pd

### Understanding the data available

In [3]:
snap_counts = nfl.import_snap_counts([2024])
print(snap_counts.columns)
#print(snap_counts['player'].head())

ids = nfl.import_ids()
print(ids.columns)
stfrd=ids[ids['gsis_id']=='00-0026498']
pfr = stfrd['pfr_id'].values[0]
snap_stfrd = snap_counts[snap_counts['pfr_player_id']==pfr]
print(len(snap_stfrd))

Index(['game_id', 'pfr_game_id', 'season', 'game_type', 'week', 'player',
       'pfr_player_id', 'position', 'team', 'opponent', 'offense_snaps',
       'offense_pct', 'defense_snaps', 'defense_pct', 'st_snaps', 'st_pct'],
      dtype='object')
Index(['cfbref_id', 'gsis_id', 'team', 'sleeper_id', 'age', 'merge_name',
       'fleaflicker_id', 'nfl_id', 'stats_id', 'name', 'draft_ovr',
       'fantasypros_id', 'sportradar_id', 'rotowire_id', 'rotoworld_id',
       'draft_year', 'twitter_username', 'position', 'pff_id',
       'stats_global_id', 'cbs_id', 'swish_id', 'yahoo_id', 'draft_pick',
       'db_season', 'ktc_id', 'mfl_id', 'pfr_id', 'birthdate',
       'fantasy_data_id', 'college', 'draft_round', 'weight', 'espn_id',
       'height'],
      dtype='object')
18


## Creating the function

In [ ]:
def snap_share(player_id, year):
    if year < 2012: 
        print(f"ERROR: year ({year}) is before the first year of snap count available (2009).")
        return -1
    ids = ids = nfl.import_ids()
    player_ids=ids[ids['gsis_id']==player_id]
    if player_ids.empty:
        print("ERROR: Player {player_id} not found.")
        return -1
    name = player_ids['name'].values[0]
    pfr_id = player_ids['pfr_id'].values[0]
    snap_counts = nfl.import_snap_counts([year])
    player_snaps = snap_counts[snap_counts['pfr_player_id']==pfr_id]
    if player_snaps.empty:
        print(f"ERROR: There was no data for player {player_id} ({name}) on that year ({year}).")
        return -1
    player_snaps = player_snaps[player_snaps['week'] <= 18]
    player_snaps = player_snaps[((player_snaps['season'] <= 2020) & (player_snaps['week'] <= 17)) | (player_snaps['season'] > 2020)]
    if player_snaps['position'].values[0]=='K':
        return player_snaps['st_pct'].mean()
    return player_snaps['offense_pct'].mean()

### Testing the function

In [23]:
player_names = ["Matthew Stafford", "Jayden Daniels", "Tom Brady",
                "Davante Adams", "Ladd McConkey", "Julio Jones",
                "Christian McCaffrey", "Tyrone Tracy Jr.", "Kenyan Drake",
                "non_existent"]
player_ids = ['00-0026498', '00-0039910', '00-0019596',
              '00-0031381', '00-0039915', '00-0027944', 
              '00-0033280', '00-0039384', '00-0033118',
              '00-0000000']
for i in range(len(player_names)):
    name = player_names[i]
    id = player_ids[i]    
    print(name)
    print(f"\tPlayer's snap share 2020: {round(snap_share(id, 2020)*100,4)}%")
    print(f"\tPlayer's snap share 2024: {round(snap_share(id, 2024)*100,4)}%")

Matthew Stafford
	Player's snap share 2020: 91.5%
	Player's snap share 2024: 100.0%
Jayden Daniels
ERROR: There was no data for player (Jayden Daniels) on that year (2020).
	Player's snap share 2020: -100%
	Player's snap share 2024: 90.8824%
Tom Brady
	Player's snap share 2020: 95.875%
ERROR: There was no data for player (Tom Brady) on that year (2024).
	Player's snap share 2024: -100%
Davante Adams
	Player's snap share 2020: 85.2143%
	Player's snap share 2024: 91.7857%
Ladd McConkey
ERROR: There was no data for player (Ladd McConkey) on that year (2020).
	Player's snap share 2020: -100%
	Player's snap share 2024: 73.1875%
Julio Jones
	Player's snap share 2020: 71.2222%
ERROR: There was no data for player (Julio Jones) on that year (2024).
	Player's snap share 2024: -100%
Christian McCaffrey
	Player's snap share 2020: 77.3333%
	Player's snap share 2024: 72.25%
Tyrone Tracy Jr.
ERROR: There was no data for player (Tyrone Tracy) on that year (2020).
	Player's snap share 2020: -100%
	Play